In [1]:
from tensorflow.keras import datasets

In [2]:
(training_images, training_labels),(testing_images,testing_labels) = datasets.cifar10.load_data()

170498071/170498071 [==============================] - 23s 0us/step


In [5]:
print(training_images.shape)
print(training_labels.shape)
print(testing_images.shape)
print(testing_labels.shape)

(50000, 32, 32, 3)
(50000, 1)
(10000, 32, 32, 3)
(10000, 1)
